In [1]:
import pandas as pd
import data_util
from tqdm.notebook import tqdm
#from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from state_util import StateUtil
from tec_an import TecAn
import numpy as np
from data_util import *
from sklearn_model_hyper import *
from joblib import dump, load

import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as scs
import scikitplot as skplt

from tensorflow.keras.layers import InputLayer, BatchNormalization, GlobalMaxPool1D, Bidirectional, Dense, Flatten, Conv2D, LeakyReLU, Dropout, LSTM, GRU, Input
from tensorflow.keras import Model, Sequential
from tensorflow.keras import datasets, layers, models
from tensorflow.keras import regularizers
from keras.wrappers.scikit_learn import KerasClassifier
from imblearn.over_sampling import RandomOverSampler
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
import tensorflow.keras as keras
import random
from catboost import CatBoost

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression



from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score


In [2]:
path = "./data/"
trainX_raw, trainY_raw = load_data("", "train", path)
valXb, valYb = load_data("backtest", "train", path)
valX4, val4 = load_data("4jul21", "train", path)

X_train, y_train = trainX_raw, trainY_raw

valX, valY = valX4, val4

features = trainX_raw.shape[-1]

print(f'Train {X_train.shape} {y_train.shape}')
print(f'Val {valX.shape} {valY.shape}')


Train (4127, 16) (4127,)
Val (270, 16) (270,)


In [3]:
def avg(array):
    if (len(array) == 0):
        return 0
    return sum(array)/len(array)

In [8]:
%%time

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.pipeline import make_pipeline
from catboost import CatBoostClassifier
from sklearn.metrics import precision_score, recall_score, f1_score


classifiers = [
    LogisticRegression(),
    KNeighborsClassifier(3),
    #GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(),
    RandomForestClassifier(max_depth=50, n_estimators=100, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    CatBoostClassifier(logging_level = 'Silent'),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()
]


CPU times: user 244 µs, sys: 1 µs, total: 245 µs
Wall time: 249 µs


In [10]:

X_train, y_train = trainX_raw, trainY_raw

approved_class = []

best_scores = []

#while (len(approved_class) == 0):
while (avg(best_scores) < 0.5):
    print("======")
    approved_class = []
    best_scores = []
    for clf in classifiers:
        
        clf = make_pipeline(StandardScaler(), clf)
        #clf = make_pipeline(MinMaxScaler(), clf)
        #clf = make_pipeline(Normalizer(), clf)
        
        clf.fit(X_train, y_train)
        score = avg([
            f1_score(val4, clf.predict(valX4)),
            f1_score(valYb, clf.predict(valXb))
            ])
        print(score)
        if (score > 0.5):
            best_scores.append(score)
            approved_class.append(clf)
            print("{}".format(score))

print(f'{len(approved_class)} selected {avg(best_scores)}')

0.2527536231884058
0.45765032675971923
0.45437445407286775
0.40447150056477543
0.18029458324729847
0.33897432296421126
0.398495137964488
0.0


KeyboardInterrupt: 

In [ ]:
len(approved_class)

In [ ]:
trainX_raw, trainY_raw = load_data("", "train", path)

bValX, bValY = load_data("backtest", "train", path)
#bValX, bValY = load_data("", "train", path)

print("{} {}".format(bValX.shape, print(Counter(bValY))))

models = []
best_scores = []

for idx, model in enumerate(approved_class):
    clf = make_pipeline(StandardScaler(), model)
    score = precision_score(bValY, clf.predict(bValX))
    best_scores.append(score)
    print(f'{model} {score}')
    models.append((f"{idx}",model))
    #dump(model, 'GaussianNB') 
    eval_data(model, bValX, bValY)
    
print(f'Avg: {avg(best_scores)}')

In [ ]:
model = approved_class[1]

preds = model.predict(bValX)

print(preds)

print(precision_score(bValY, preds))

eval_data(model, bValX, bValY)


In [5]:
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

estimators = [
     ('rf',  RandomForestClassifier(max_depth=10, n_estimators=100, random_state=42)),
     ('kn',  KNeighborsClassifier()),
     ('ada', AdaBoostClassifier()),
     ('GaussianNB', GaussianNB()),
     ('SVC', SVC(kernel="linear", C=0.025)),
     #('SVC2', SVC(gamma=2, C=1)),
     #('QuadraticDiscriminantAnalysis', QuadraticDiscriminantAnalysis()),
     ('mlp', MLPClassifier(alpha=1, max_iter=1000)),
     ('cat', CatBoostClassifier(logging_level='Silent'))
 ]

clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
    #estimators=estimators, final_estimator=MLPClassifier()
)

clf = make_pipeline(StandardScaler(), clf)


In [6]:
%%time

X_train, y_train = trainX_raw, trainY_raw

#X_test, y_test = bValX, bValY = load_data("backtest", "train", path)

clf.fit(X_train, y_train)

eval_data(clf, X_test, y_test)

NameError: name 'X_test' is not defined

In [7]:
X_test, y_test = bValX, bValY = load_data("backtest", "train", path)

eval_data(clf, X_test, y_test)

              precision    recall  f1-score   support

     class 0       0.55      0.68      0.61       985
     class 1       0.49      0.36      0.41       840

    accuracy                           0.53      1825
   macro avg       0.52      0.52      0.51      1825
weighted avg       0.52      0.53      0.52      1825



In [ ]:
from joblib import dump, load
#dump(clf, 'model/StackingClassifier') 

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

#classifiers

clf = VotingClassifier(estimators=estimators, voting='hard')

#clf = make_pipeline(StandardScaler(), clf)

clf.fit(X_train, y_train)

eval_data(clf, X_test, y_test)

In [ ]:
from joblib import dump, load

#dump(model, 'model/Keras') 